# RAG Evaluation Dataset Generation

In [2]:
import os
import datetime
import pandas as pd
import json
import random
from sqlalchemy import make_url
import nest_asyncio
nest_asyncio.apply() 

from llama_index.core.llama_dataset.generator import RagDatasetGenerator
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.vector_stores.postgres import PGVectorStore
from llama_index.core.vector_stores.types import VectorStoreQuery
from llama_index.llms.openai_like import OpenAILike

import sys
utils_path = "../../../utils"
if utils_path not in sys.path:
    sys.path.append(utils_path)

from helpers import (
    remove_recs_without_query,
    remove_duplicated_queries,
)

In [3]:
# PGVector DB params
DB_PORT = 5432
DB_USER = "demouser"
DB_PASSWD = "demopasswd"
DEFAULT_DB = "postgres"
DB_NAME = "vectordb"
DB_HOST = "localhost"
TABLE_NAME = "NASA_HISTORY_BOOKS"
CONNECT_STRING = f"postgresql://{DB_USER}:{DB_PASSWD}@{DB_HOST}:{DB_PORT}/{DEFAULT_DB}"

# LLamaIndex embedding model
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-base-en-v1.5",
)
EMBEDDING_SIZE = len(Settings.embed_model.get_text_embedding("hi"))

# LLM service settings
MIN_RESPONSE_LENGTH = 50

# LLM service settings
LLM_MODEL = "mistralai/Mixtral-8x7B-Instruct-v0.1" # Use the most powerful LLM at hand
LLM_API_BASE = "vLLM_API_BASE" # Chenge this value with you'r URL
LLM_API_KEY = "NO_KEY" # vLLM does not require a key by default
GEN_TEMP=0.1    # Temperature generation
MAX_TOKENS=1024 # Max, number of tokens to generate
REP_PENALTY=1.03 # Word repetition penalty at generation time

# Initialize LLamaIndex LLM provider
Settings.llm = OpenAILike(
    model=LLM_MODEL,
    api_key=LLM_API_KEY,
    api_base=LLM_API_BASE,
    temperature=GEN_TEMP,
    max_tokens=MAX_TOKENS,
    repetition_penalty=REP_PENALTY,
)

# Number of workers for the dataset generator
NUM_WORKERS = 4

In [5]:
# Directory path to save the Q/A files
SAVE_DIR = f"../qa_datasets/{LLM_MODEL.split('/')[-1]}"
if os.path.isdir(SAVE_DIR):
    now = '_'.join(f"{datetime.datetime.now()}".split('.')[0].split())
    os.rename(SAVE_DIR, f"{SAVE_DIR}-{now}")
os.mkdir(SAVE_DIR)
print(f"Saving evaluation datasets at {SAVE_DIR}")

Saving evaluation datasets at ../qa_datasets/Mixtral-8x7B-Instruct-v0.1


In [6]:
%%time

# Open the connection to the pre-populated PGVector store
url = make_url(CONNECT_STRING)
vector_store = PGVectorStore.from_params(
    database=DB_NAME,
    host=url.host,  
    password=url.password,
    port=url.port,
    user=url.username,
    table_name=TABLE_NAME,
    embed_dim=EMBEDDING_SIZE, # embedding model dimension
    cache_ok=True,
    hybrid_search=True,
)

# Load all nodes from the vector store.  Adjust `similarity_top_k` to be >= number of nodes in the vector store
query = VectorStoreQuery(query_embedding=[0 for i in range(EMBEDDING_SIZE)], similarity_top_k=10_000_000)
result = vector_store.query(query)
nodes = result.nodes
print(f"Loaded {nodes} nodes from the vector store")

CPU times: user 851 ms, sys: 110 ms, total: 961 ms
Wall time: 1.09 s


In [8]:
# Set the number of Q/A pairs to generate at 10% of the total nodes
num_eval_nodes = int(round(len(nodes)*0.1, 0))
eval_nodes_sample = random.sample(nodes, num_eval_nodes)

In [9]:
%%time
# Initialize the dataset generator
dataset_generator = RagDatasetGenerator(
    nodes=eval_nodes_sample,
    show_progress=True,
    num_questions_per_chunk=1,
    workers=NUM_WORKERS
)

# Generate the Q/A pairs from nodes
rag_eval_dataset =  dataset_generator.generate_dataset_from_nodes() 

100%|██████████| 4/4 [00:05<00:00,  1.48s/it]

CPU times: user 39 s, sys: 4.13 s, total: 43.1 s
Wall time: 1h 18min 50s


In [14]:
# Adjust Pandas siaplay settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Set the bar for the minimum length (words) for an answer in a Q/A pair
MIN_RESPONSE_LENGHT = 100

# Convert the test set to Pandas
rag_eval_df = rag_eval_dataset.to_pandas()

# Keep real questions only
rag_eval_df = rag_eval_df[rag_eval_df["query"].str.contains("\?")]

# Filter out too short responses
too_short_responses = rag_eval_df["reference_answer"].str.count(' ') < MIN_RESPONSE_LENGHT
rag_eval_df = rag_eval_df[~too_short_responses]
clean_rag_eval_df = rag_eval_df.drop_duplicates(
    subset=["query"],
    ignore_index=True,
).copy()

# Serialize the dataset as CSV file
clean_rag_eval_df.to_csv(
    path_or_buf=f"{SAVE_DIR}/NASA_history_qa_only.csv",
    index=False,
)

In [16]:
# Display some examples
clean_rag_eval_df[['query', 'reference_answer', 'reference_contexts']].sample(20)

,query,reference_answer,reference_contexts
57,What was the significance of propeller cuffs in the refinement of aircraft propeller blade design and how did they contribute to the performance of the North American P-51 Mustang fighter and the cooling of radial engine designs such as the Republic P-47 Thunderbolt and the Boeing B-29 Superfortress long-range bomber?,"\nPropeller cuffs were significant in the refinement of aircraft propeller blade design as they allowed for the modification of existing blade designs, particularly in increasing the blade area. This modification enabled aircraft to reach higher speeds, as demonstrated by a single-engine fighter that was theoretically capable of 400 mph with the right propeller but could not yet reach that speed with a conventional propeller root-shape.\n\nThe North American P-51 Mustang fighter was modified with blade cuffs for increased performance, which maximized the aircraft's overall performance. Similarly, cuffed propellers improved the cooling of radial engine designs such as the Republic P-47 Thunderbolt and the four-engine Boeing B-29 Superfortress long-range bomber, which were known for their notorious cooling and engine fire problems.\n\nTherefore, propeller cuffs played a crucial role in enhancing the performance of fighter aircraft and improving the cooling of radial engine designs, contributing to their overall efficiency and safety.","[The NACA and Aircraft Propulsion, 1915–1958\n11\nNACA researchers also worked to refine the aerodynamic properties of a \npropeller blade along its entire length with airfoil sections called cuffs. Blade \ndesign reflected a compromise where most of the blade was an airfoil, but the \nportion where it attached to the hub, called the root, was round for structural \nstrength. In 1939, Langley researchers in the Full-Scale Tunnel investigated a \nsingle-engine fighter that was theoretically capable of 400 mph with the right \npropeller but could not yet reach that speed with a conventional propeller \nroot-shape. To each blade root, they attached airfoil-shaped cuffs that covered \nover 45 percent of the blade. The increased blade area enabled the fighter to \nreach 400 mph at 20,000 feet. Langley carried on with extensive research on \ncuffs that allowed for the modification of existing blade designs.49 The North \nAmerican P-51 Mustang fighter was modified with blade cuffs for increased \nperformance, and cuffed propellers likewise improved the cooling of radial \nengine designs such as the Republic P-47 Thunderbolt and—very significantly, \nbecause of its notorious cooling and engine fire problems—the four-engine \nBoeing B-29 Superfortress long-range bomber.\nFigure 1-2. The use of propeller cuffs on fighters like the P-51 Mustang maximized the \naircraft’s overall performance. (National Air and Space Museum, Smithsonian Institution, \nNASM 7A35592)]"
11,Question: What are the features of the ladies' dressing room on the aircraft?,"\nThe ladies' dressing room on the aircraft is furnished in the modern style and equipped with a hand lavatory with hot and cold running water, a dental lavatory, a large double dressing table complete with four lights, two large plateglass mirrors, two benches, receptacles for Kleenex, etc. A ladies' toilet compartment opens from this dressing room, and each toilet is equipped with a standard size toilet seat and is of the flushing type. The flushing water is the waste water from the hand and dental lavatories. As the toilet top is raised, a cylinder is filled with this water and when the lid is lowered, this cylinder discharges the water, flushing the toilet bowl, and then the contents are dumped outside the aircraft.","[347\nDocument 5-23 (a–c)\nfoodstuffs and galley supplies is 256 pounds[,] making a total of 490 pounds. In \naddition, there are two drinking fountains, one on each end of the passengers’ deck. \nIncidentally, the galley equipment was designed and built by Pan American Airways.\nThe ladie

In [17]:
# Serialize the raw dataset as a JSON file
rag_eval_dataset.save_json(
    path=f"{SAVE_DIR}/NASA_history_qa_not_clean.json"
)

# Deduplicate, filter non-queries and serialize the dataset as JSON file
dataset_dict = json.loads(rag_eval_dataset.json())
dataset_dict['examples'] = remove_recs_without_query(
    dataset_dict['examples']
)
dataset_dict['examples'] = remove_duplicated_queries(
    dataset_dict['examples']
)

# Serialize the JSON object.
with open(f"{SAVE_DIR}/NASA_history_qa_only.json" , "w") as write:
    json.dump(dataset_dict , write)